# Loading big datasets

In this series of notebooks, we will analyze the nutritional profile of food products using an open database : OpenFoodFacts.

## OpenFoodFacts
[OpenFoodFacts](https://fr-en.openfoodfacts.org/) is an open database on food products.
It is built as a common, where anyone can contribute data on packaged food products : pictures, ingredients, nutritional values, etc.
This database serves as the basis for many apps for mobile phones, in particular scan apps to help customers when they buy groceries.

The database and various exports can be downloaded from [this page](https://fr-en.openfoodfacts.org/data).

We will work on the CSV export of the database.
It contains data on more than 1.4 million products and weighs 3 GB (as of 2020-09-26).

You need to :
- Download the data from [this link](https://static.openfoodfacts.org/data/fr.openfoodfacts.org.products.csv).
- Open the documentation of the CSV export, with useful information about the fields, in this [text file](https://static.openfoodfacts.org/data/data-fields.txt).

In [4]:
# change to the (absolute or relative) path to the CSV file on your computer
CSV_FILE = '/home/mathieu/datasets/openfoodfacts/2020-09-26/fr.openfoodfacts.org.products.csv'

## Testing the waters : Requirements before loading

If we naively call `read_csv` with its default parameters on a file as big as the CSV export from OpenFoodFacts, there is a chance that things don't go well and pandas completely fills the computer memory before crashing.
This is due to the fact that [the CSV file format is not a fully standardized format](https://en.wikipedia.org/wiki/Comma-separated_values).
Rather, CSV is currently used as an umbrella term for a family of formats for flat, tabular data.

The documentation of [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) and its list of parameters give a broad overview of the main variants of the format, such as which character is used as the field separator (or delimiter), whether column names are written as a header of one or more lines, how to handle and interpret blank lines, how to handle and interpret quoted items, etc.
`read_csv` tries to be smart and infer some of these properties but does not always succeed, so you need to guide it and provide it with the right parameters for your file.

You can usually find out more about the format of your dataset by reading the accompanying documentation.
Utilities such as [csvkit](https://csvkit.readthedocs.io/en/latest/) can help you figuring out the rest.
Last, a simple pager program such as [less](https://en.wikipedia.org/wiki/Less_(Unix)) is very helpful when you deal with very large files.

For the CSV export of OpenFoodFacts :
- The field separator in this file is a `<tab>`, as stated in the documentation (text file linked above) ;
- The first line contains the column names, as can be seen when opening the file with `less`.

Hence we will need to provide `read_csv` with parameters :
- `sep='\t'`,
- `header=0` (the first line has index 0, because python counts from 0).

## Dipping our toes : Loading slices of the dataset

In [2]:
import pandas as pd

We start by reading only the first 1000 rows (`nrows=1000`) to have a first look at the dataset and check its memory usage.
Then we read the first 7000 rows, then the first 14000 rows, and monitor memory usage as a function of the number of rows.
We will then perform a rough back-of-the-envelope calculation to extrapolate the memory requirements for loading the entire dataset (> 1.4 million entries), and evaluate the need for applying a few optimizations.

By default, [info()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) returns a *shallow* computation of memory usage.
Specifying `memory_usage='deep'` triggers a deeper introspection that returns a better estimate of the memory usage.

In [14]:
# read the first 1000 rows, display the first 5 rows
df = pd.read_csv(CSV_FILE, sep='\t', header=0, nrows=1000)
df.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,17,http://world-fr.openfoodfacts.org/produit/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31,http://world-fr.openfoodfacts.org/produit/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3327986,http://world-fr.openfoodfacts.org/produit/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,http://world-fr.openfoodfacts.org/produit/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1111111111,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check the memory usage of the DataFrame containing these 1000 rows.

In [15]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 181 entries, code to carnitine_100g
dtypes: float64(123), int64(3), object(55)
memory usage: 4.8 MB


Load the first 7000 rows and check their memory usage.

In [24]:
# load 7000 rows and check memory usage


Load the first 14000 rows and check their memory usage.

In [54]:
# load 14000 rows and check memory usage


Can you observe a trend in the memory usage ?
How much memory would be necessary to load 1.4 million entries ?

## Helper 1 : Loading only the columns of interest
[read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) has a parameter `usecols` where you can specify which columns you want to load.
The values for the other columns are discarded immediately after they are read, which directly reduces memory usage.
This seems like a good, safe, first strategy to cut memory usage.

It is often easier to determine the columns that we *do not want to keep*.
They fall under two categories :
- Columns that are not directly useful for our primary purpose of analyzing nutritional values, such as those containing metadata for the database entry ("creator", "last_modified_t", etc.), manufacturing places, place of purchase, etc ;
- Columns that are completely, or almost completely empty, and will not be able to contribute to the analysis.

It is likely that many columns in OpenFoodFacts will belong to the second category, because the database schema contains many columns for data that is not mandatory on food packaging, or data that is harder to collect.

In order to check the emptiness of columns, we will once again make use of `info`.
Because there are many columns, `info` only displays a minimal summary by default, so we need to explicitly ask for the full list of column names, types and non-null counts with the parameters `verbose=True, null_counts=True`.

In [28]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 181 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   code                                        14000 non-null  object 
 1   url                                         14000 non-null  object 
 2   creator                                     14000 non-null  object 
 3   created_t                                   14000 non-null  int64  
 4   created_datetime                            14000 non-null  object 
 5   last_modified_t                             14000 non-null  int64  
 6   last_modified_datetime                      14000 non-null  object 
 7   product_name                                13641 non-null  object 
 8   generic_name                                195 non-null    object 
 9   quantity                                    875 non-null    object 
 10  packaging

As you can see :
- Some fields are always set,
- A number of fields are often set,
- Many columns are never set.

Optional exercise : plot the distribution of column non-null counts.

Once we exclude the metadata and (mostly) empty columns, we are left with the set of columns that will be useful for our analysis.
Here we just filter the columns in the current dataset, but in the next notebook we will provide `read_csv` with this list of columns.

In [64]:
keep_cols = ["code", "url", "product_name", "brands", "categories",
             "countries_tags",
             "additives_tags",
             "nutriscore_score", "nutriscore_grade",
             "nova_group",
             "pnns_groups_1", "pnns_groups_2",
             "states",
             "energy-kcal_100g",
             "fat_100g", "saturated-fat_100g", "trans-fat_100g", "cholesterol_100g",
             "carbohydrates_100g", "sugars_100g",
             "fiber_100g",
             "proteins_100g",
             "salt_100g", "sodium_100g",
             "vitamin-a_100g",
             "vitamin-c_100g",
             "calcium_100g",
             "iron_100g",
             "nutrition-score-fr_100g"
]
# check that our list of columns is correct and the selection works
df = df[keep_cols]
df

This filtering on columns can be done during reading for greater efficiency.

In [66]:
df = pd.read_csv(CSV_FILE, sep='\t', header=0, usecols=keep_cols, nrows=14000)

Loading only 29 columns instead of the original 181 columns should have saved a lot of memory.

Check the (deep) memory usage of this smaller DataFrame.

In [30]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   code                     14000 non-null  object 
 1   url                      14000 non-null  object 
 2   product_name             13641 non-null  object 
 3   brands                   9562 non-null   object 
 4   categories               10649 non-null  object 
 5   countries_tags           13964 non-null  object 
 6   additives_tags           7778 non-null   object 
 7   nutriscore_score         8301 non-null   float64
 8   nutriscore_grade         8301 non-null   object 
 9   nova_group               10825 non-null  float64
 10  pnns_groups_1            13980 non-null  object 
 11  pnns_groups_2            13996 non-null  object 
 12  states                   14000 non-null  object 
 13  energy-kcal_100g         12844 non-null  float64
 14  fat_100g              

Using the same ratio as earlier, how much memory would be needed to load the full dataset restricted to these 29 columns ?

## Helper 2 : Explicitly modelling categorical variables
Some columns in the OpenFoodFacts dataset correspond to categorical variables, for example the [NOVA groups](https://world.openfoodfacts.org/nova) (`nova_group`) and [Nutri-Score grade](https://en.wikipedia.org/wiki/Nutri-score) (`nutriscore_grade`).
Categorical variables typically have a small number of possible values, much lower than the number of occurrences in a big dataset.

pandas has a specific data type named [categorical](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) with 2 main advantages here :
- memory efficiency : all instances of a given value point to a unique shared representation, [saving memory](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=categorical#memory-usage),
- interoperability : categorical variables have specific support in libraries for statistical analysis, machine learning, or visualization (plots).

In [44]:
from pandas.api.types import CategoricalDtype

We can search for candidate categorical variables by looking at their name and the number of unique values in each column of the dataset.

Display the number of unique value in each column of the DataFrame with [nunique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html).

In [79]:
# TODO


Four columns have a very low number of distinct values, and a name that could correspond to categorical variables :
- `nutriscore_grade`,
- `nova_group`,
- `pnns_groups_1`,
- `pnns_groups_2`.

We can check if this is the case by looking at the list of their unique values.

List the unique values for `nutriscore_grade` using [unique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html).

In [80]:
# TODO


The 6 values of `nutriscore_grade` correspond to the 5 values of the [Nutri-Score](https://en.wikipedia.org/wiki/Nutri-score), plus `nan` for missing values.
This is indeed a categorical variable.

List the unique values for `nova_group`.

In [81]:
# TODO


List the unique values for `pnns_groups_1`.

In [82]:
# TODO


List the unique values for `pnns_groups_2`.

In [83]:
# TODO


Categorical variables can be :
- unordered, like `pnns_groups_1` and `pnns_groups_2`,
- ordered, like `nova_group` (from '1' for unprocessed foods to '4' for ultra-processed foods) and `nutriscore_grade` (from 'a' for the healthiest foods to 'e' for the least healthy foods).

Declare those categorical variables in a mapping that can serve as the argument `dtype` of `read_csv`.

In [75]:
dtype = {
    # 'code' should be read as a string
    'code': str,
    # ordered categoricals, with explicit list of values
    'nova_group': CategoricalDtype(categories=['1', '2', '3', '4'], ordered=True),
    'nutriscore_grade': CategoricalDtype(categories=['a', 'b', 'c', 'd', 'e'], ordered=True),
    # unordered categoricals, values will be inferred during reading
    'pnns_groups_1': 'category',
    'pnns_groups_2': 'category'
}

Re-load the first 14000 entries of the dataset, restricted to `keep_cols`, with explicit data types for categorical variables.

In [76]:
df = pd.read_csv(CSV_FILE, sep='\t', header=0, usecols=keep_cols, dtype=dtype, nrows=14000)
df

,code,url,product_name,brands,categories,countries_tags,additives_tags,nutriscore_score,nutriscore_grade,nova_group,...,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g
0,0000000000017,http://world-fr.openfoodfacts.org/produit/0000...,Vitória crackers,NaN,NaN,en:france,NaN,NaN,NaN,NaN,...,15.00,NaN,7.80,1.4000,0.560,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-fr.openfoodfacts.org/produit/0000...,Cacao,NaN,NaN,en:france,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-fr.openfoodfacts.org/produit/0000...,Filetes de pollo empanado,NaN,NaN,en:spain,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-fr.openfoodfacts.org/produit/0000...,moutarde au moût de raisin,courte paille,"Epicerie, Condiments, Sauces, Moutardes",en:france,NaN,18.0,d,NaN,...,22.00,0.0,5.10,4.6000,1.840,NaN,NaN,NaN,NaN,18.0
4,00000000001111111111,http://world-fr.openfoodfacts.org/produit/0000...,Sfiudwx,Watt,Xsf,en:france,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,0011213902654,http://world-fr.openfoodfacts.org/produit/0011...,Deli Style Roast Beef,Spartan,"Meats, Prepared meats",en:united-states,"en:e150c,en:e262,en:e262ii,en:e326",4.0,c,4,...,0.00,0.0,30.36,1.5625,0.625,0.000000,0.0000,0.000,0.00257,4.0
13996,0011213902685,http://world-fr.openfoodfacts.org/produit/0011...,"Spartan, original diced tomato with green chilies",Spartan,"Plant-based foods and beverages, Plant-based f...",en:united-states,en:e330,-3.0,a,4,...,2.48,0.8,0.83,0.6200,0.248,0.000124,0.0050,0.033,0.00060,-3.0
13997,0011213902692,http://world-fr.openfoodfacts.org/produit/0011...,"Spartan, diced pears",Spartan,"Plant-based foods and beverages, Plant-based f...",en:united-states,en:e330,-3.0,a,3,...,13.27,0.9,0.00,0.0225,0.009,0.000000,0.0531,0.000,0.00000,-3.0
13998,0011213902722,http://world-fr.openfoodfacts.org/produit/0011...,"Spartan, hot cocoa mix, milk chocolate",NaN,"Beverages, Dried products, Dried products to b...",en:united-states,"en:e100,en:e340,en:e340ii,en:e471,en:e551",NaN,NaN,4,...,80.00,2.9,5.71,1.4275,0.571,0.000000,0.0000,0.171,0.00206,NaN


In [78]:
# check the DataFrame
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   code                     14000 non-null  object  
 1   url                      14000 non-null  object  
 2   product_name             13641 non-null  object  
 3   brands                   9562 non-null   object  
 4   categories               10649 non-null  object  
 5   countries_tags           13964 non-null  object  
 6   additives_tags           7778 non-null   object  
 7   nutriscore_score         8301 non-null   float64 
 8   nutriscore_grade         8301 non-null   category
 9   nova_group               10825 non-null  category
 10  pnns_groups_1            13980 non-null  category
 11  pnns_groups_2            13996 non-null  category
 12  states                   14000 non-null  object  
 13  energy-kcal_100g         12844 non-null  float64 
 14  fat_10

## Helper 3 : Reducing the precision of floating-point numbers

Another strategy that helps to reduce memory usage is to specify data types that suit the actual level of precision of the values.
By default, pandas stores floating-point numbers as `float64`, ie. [Double-precision floating-point format](https://en.wikipedia.org/wiki/Double-precision_floating-point_format).

On the OpenFoodfacts dataset, this conservative approach could be considered wasteful because nutritional values written on food packaging are not of an extreme precision.
The minor loss of precision should not affect dramatically the analysis, so we could replace `float64` with [float16](https://en.wikipedia.org/wiki/Half-precision_floating-point_format) and cut memory usage for these columns by a factor of 4.

Cast the type of columns whose name ends with `_100g` as `float16` using [astype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html).

In [38]:
num_cols = [x for x in list(df.columns) if x.endswith("_100g")]
dtype = {x: "float16" for x in num_cols}
df = df.astype(dtype)
df.head(5)

,code,url,product_name,brands,categories,countries_tags,additives_tags,nutriscore_score,nutriscore_grade,nova_group,...,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g
0,0000000000017,http://world-fr.openfoodfacts.org/produit/0000...,Vitória crackers,NaN,NaN,en:france,NaN,NaN,NaN,NaN,...,15.0,NaN,7.800781,1.400391,0.560059,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-fr.openfoodfacts.org/produit/0000...,Cacao,NaN,NaN,en:france,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-fr.openfoodfacts.org/produit/0000...,Filetes de pollo empanado,NaN,NaN,en:spain,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-fr.openfoodfacts.org/produit/0000...,moutarde au moût de raisin,courte paille,"Epicerie, Condiments, Sauces, Moutardes",en:france,NaN,18.0,d,NaN,...,22.0,0.0,5.101562,4.601562,1.839844,NaN,NaN,NaN,NaN,18.0
4,00000000001111111111,http://world-fr.openfoodfacts.org/produit/0000...,Sfiudwx,Watt,Xsf,en:france,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Do you notice any change in the floating-point values ?

Check memory usage.

In [39]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   code                     14000 non-null  object 
 1   url                      14000 non-null  object 
 2   product_name             13641 non-null  object 
 3   brands                   9562 non-null   object 
 4   categories               10649 non-null  object 
 5   countries_tags           13964 non-null  object 
 6   additives_tags           7778 non-null   object 
 7   nutriscore_score         8301 non-null   float64
 8   nutriscore_grade         8301 non-null   object 
 9   nova_group               10825 non-null  float64
 10  pnns_groups_1            13980 non-null  object 
 11  pnns_groups_2            13996 non-null  object 
 12  states                   14000 non-null  object 
 13  energy-kcal_100g         12844 non-null  float16
 14  fat_100g              

## Conclusion
We should now be able to read the entire dataset by specifying the subset of columns that we are actually interested in, and providing a data type mapping for these columns.